In [13]:
import numpy as np
import pandas as pd


from sklearn.base import BaseEstimator, ClassifierMixin

from sklearn.utils import check_random_state

from sklearn.preprocessing import LabelEncoder

# Dataset:

In [14]:
data = pd.read_csv("data.csv")
dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset
Y = dataset
X = X[:, 0:2500]
Y = Y[:, 2500]


In [15]:
def projection_simplex(v, z=1):

    """

    Projection onto the simplex:

        w^* = argmin_w 0.5 ||w-v||^2 s.t. \sum_i w_i = z, w_i >= 0

    """

    # For other algorithms computing the same projection, see

    # https://gist.github.com/mblondel/6f3b7aaad90606b98f71

    n_features = v.shape[0]

    u = np.sort(v)[::-1]

    cssv = np.cumsum(u) - z

    ind = np.arange(n_features) + 1

    cond = u - cssv / ind > 0

    rho = ind[cond][-1]

    theta = cssv[cond][-1] / float(rho)

    w = np.maximum(v - theta, 0)

    return w

In [16]:
class MulticlassSVM(BaseEstimator, ClassifierMixin):



    def __init__(self, C=1, max_iter=50, tol=0.05,

                 random_state=None, verbose=0):

        self.C = C

        self.max_iter = max_iter

        self.tol = tol,

        self.random_state = random_state

        self.verbose = verbose



    def _partial_gradient(self, X, y, i):

        # Partial gradient for the ith sample.

        g = np.dot(X[i], self.coef_.T) + 1

        g[y[i]] -= 1

        return g



    def _violation(self, g, y, i):

        # Optimality violation for the ith sample.

        smallest = np.inf

        for k in range(g.shape[0]):

            if k == y[i] and self.dual_coef_[k, i] >= self.C:

                continue

            elif k != y[i] and self.dual_coef_[k, i] >= 0:

                continue



            smallest = min(smallest, g[k])



        return g.max() - smallest



    def _solve_subproblem(self, g, y, norms, i):

        # Prepare inputs to the projection.

        Ci = np.zeros(g.shape[0])

        Ci[y[i]] = self.C

        beta_hat = norms[i] * (Ci - self.dual_coef_[:, i]) + g / norms[i]

        z = self.C * norms[i]



        # Compute projection onto the simplex.

        beta = projection_simplex(beta_hat, z)



        return Ci - self.dual_coef_[:, i] - beta / norms[i]



    def fit(self, X, y):

        n_samples, n_features = X.shape



        # Normalize labels.

        self._label_encoder = LabelEncoder()

        y = self._label_encoder.fit_transform(y)



        # Initialize primal and dual coefficients.

        n_classes = len(self._label_encoder.classes_)

        self.dual_coef_ = np.zeros((n_classes, n_samples), dtype=np.float64)

        self.coef_ = np.zeros((n_classes, n_features))



        # Pre-compute norms.

        norms = np.sqrt(np.sum(X ** 2, axis=1))



        # Shuffle sample indices.

        rs = check_random_state(self.random_state)

        ind = np.arange(n_samples)

        rs.shuffle(ind)



        violation_init = None

        for it in range(self.max_iter):

            violation_sum = 0



            for ii in range(n_samples):

                i = ind[ii]



                # All-zero samples can be safely ignored.

                if norms[i] == 0:

                    continue



                g = self._partial_gradient(X, y, i)

                v = self._violation(g, y, i)

                violation_sum += v



                if v < 1e-12:

                    continue



                # Solve subproblem for the ith sample.

                delta = self._solve_subproblem(g, y, norms, i)



                # Update primal and dual coefficients.

                self.coef_ += (delta * X[i][:, np.newaxis]).T

                self.dual_coef_[:, i] += delta



            if it == 0:

                violation_init = violation_sum



            vratio = violation_sum / violation_init



            if self.verbose >= 1:

                print("iter", it + 1, "violation", vratio)



            if vratio < self.tol:

                if self.verbose >= 1:

                    print("Converged")

                break



        return self



    def predict(self, X):

        decision = np.dot(X, self.coef_.T)

        pred = decision.argmax(axis=1)

        return self._label_encoder.inverse_transform(pred)



In [17]:
data.head()

,pixel_0000,pixel_0001,pixel_0002,pixel_0003,pixel_0004,pixel_0005,pixel_0006,pixel_0007,pixel_0008,pixel_0009,...,pixel_2491,pixel_2492,pixel_2493,pixel_2494,pixel_2495,pixel_2496,pixel_2497,pixel_2498,pixel_2499,emoji
0,0,0,3,0,255,254,255,253,255,1,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,1,12,139,254,255,255,253,...,0,0,0,0,0,0,0,0,0,1
2,0,2,0,3,255,253,255,255,255,255,...,0,0,0,0,0,0,0,0,0,1
3,0,0,50,255,255,250,255,255,255,255,...,48,2,0,0,0,1,0,0,0,1
4,1,253,254,255,255,255,186,0,0,0,...,252,145,0,0,0,2,0,0,0,1


In [18]:
clf = MulticlassSVM(C=0.1, tol=0.01, max_iter=100, random_state=0, verbose=1)

clf.fit(X, Y)

print(clf.score(X, Y))

iter 1 violation 1.0
iter 2 violation 0.901983596810397
iter 3 violation 0.6213369079021905
iter 4 violation 0.4487780244504612
iter 5 violation 0.34769918928957766
iter 6 violation 0.28042406841551404
iter 7 violation 0.23657946437416738
iter 8 violation 0.20957720163060492
iter 9 violation 0.18735532848771788
iter 10 violation 0.16257023650133132
iter 11 violation 0.14572920041517062
iter 12 violation 0.13457096627159879
iter 13 violation 0.12433282478283128
iter 14 violation 0.11475858715704047
iter 15 violation 0.10581476252048275
iter 16 violation 0.09774408711921459
iter 17 violation 0.09091730848971058
iter 18 violation 0.08626004070874214
iter 19 violation 0.08113328747333831
iter 20 violation 0.07605079775756403
iter 21 violation 0.07267325555815
iter 22 violation 0.06800489239041356
iter 23 violation 0.06383998222346339
iter 24 violation 0.060240119368989105
iter 25 violation 0.05797951986346733
iter 26 violation 0.05557364346982968
iter 27 violation 0.05376468895970453
iter 